option 1: spark

In [2]:
base_path = "/mnt/processed_data_criminal_case_analysis/drug_related_DrugTypeAmount_Penalty_Location_March_19"

output_path = "/mnt/processed_data_criminal_case_analysis/drug_related_DrugTypeAmount_Penalty_Location_DataframeAlignment_March_20"

from pyspark.sql.functions import lit

# Assuming `common_columns` is a list of all columns identified
def align_dataframe(df, common_columns):
    for col in common_columns:
        if col not in df.columns:
            df = df.withColumn(col, lit(None))
    return df.select(common_columns)

common_columns = [
    "OriginalLink",
    "CaseNumber",
    "CaseName",
    "Court",
    "Location",
    "CaseType",
    "TrialProcedure",
    "JudgmentDate",
    "PublicationDate",
    "PartiesInvolved",
    "CausesofAction",
    "LegalBasis",
    "FullText",
    "drug_a",
    "amount_a",
    "drug_b",
    "amount_b",
    "drug_c",
    "amount_c",
    "ResponseText",
    "Charge1forCriminalA",
    "FineforCriminalA",
    "TotalImprisonmentLengthforCriminalA",
    "SuspendedforCriminalA",
    "Charge2forCriminalA",
    "Charge1forCriminalB",
    "Charge2forCriminalB",
    "FineforCriminalB",
    "TotalImprisonmentLengthforCriminalB",
    "SuspendedforCriminalB",
    "Charge1forCriminalC",
    "Charge2forCriminalC",
    "FineforCriminalC",
    "TotalImprisonmentLengthforCriminalC",
    "SuspendedforCriminalC",
    "Charge1forCriminalD",
    "Charge2forCriminalD",
    "FineforCriminalD",
    "TotalImprisonmentLengthforCriminalD",
    "SuspendedforCriminalD",
    "Charge1forCriminalE",
    "Charge2forCriminalE",
    "FineforCriminalE",
    "TotalImprisonmentLengthforCriminalE",
    "SuspendedforCriminalE",
    "Charge1forCriminalF",
    "Charge2forCriminalF",
    "FineforCriminalF",
    "TotalImprisonmentLengthforCriminalF",
    "SuspendedforCriminalF",
    "Charge3forCriminalC",
    "Province",
    "City",
    "District",
    "CourtLevel",
    "Adcode"
]


56


In [ ]:
sub_files = dbutils.fs.ls(base_path)

for sub_file in sub_files:
    if sub_file.name.endswith(".csv"):
        file_path = sub_file.path
        df = spark.read.format("csv") \
            .option("header", "true") \
            .option("inferSchema", "true") \
            .load(file_path)
        df_aligned = align_dataframe(df, common_columns)
        df_aligned.write.mode("overwrite").option("header", "true").csv(output_path)
        
        print(f"Aligned dataframe for {file_path}")
        df_aligned.printSchema()
        

option 2: spark sql=

In [ ]:
# Register the DataFrame as a temp view
df.createOrReplaceTempView("cases_view")

# Execute SQL query to find non-integer values
non_integer_values_query = """
SELECT OriginalLink, CaseNumber,	CaseName,	Court,	Location,	CaseType,	TrialProcedure,	JudgmentDate,PublicationDate, TotalImprisonmentLengthforCriminalA, PartiesInvolved,	CausesofAction,	LegalBasis,	FullText,	drug_a,	amount_a,	drug_b,	amount_b,	ResponseText,	Charge1forCriminalA	FineforCriminalA,	TotalImprisonmentLengthforCriminalA,	SuspendedforCriminalA,	Charge2forCriminalA,	Charge1forCriminalB,	Charge2forCriminalB	FineforCriminalB,	TotalImprisonmentLengthforCriminalB,	SuspendedforCriminalB,	Province,	City,	District,	CourtLevel,	Adcode
FROM cases_view
WHERE CAST(TotalImprisonmentLengthforCriminalA AS INT) IS NULL
AND TotalImprisonmentLengthforCriminalA IS NOT NULL
"""

non_integer_values = spark.sql(non_integer_values_query)

# Show the results
non_integer_values.show()


In [ ]:
from pyspark.sql.functions import col, isnan, when, count

# Assuming 'df' is your DataFrame
# Replace 'TotalImprisonmentLengthforCriminalA' with the actual column name you want to check
# Repeat the process for other TotalImprisonmentLength columns as needed

# Try casting the column to an integer type
df_with_cast = df.withColumn("TotalImprisonmentLengthInt", col("TotalImprisonmentLengthforCriminalB").cast("int"))

# Filter to find rows where cast is not successful
# This condition checks for nulls in the casted column which indicates unsuccessful casts
non_integer_rows = df_with_cast.filter(df_with_cast["TotalImprisonmentLengthInt"].isNull() & ~df_with_cast["TotalImprisonmentLengthforCriminalB"].isNull())

# Show the rows with non-integer values
non_integer_rows.select("TotalImprisonmentLengthforCriminalB").show(100, truncate=False)
#non_integer_rows.show(non_integer_rows.count(), truncate=False)


option 3: pd

In [3]:
columns_to_drop = [
    "drug_c",
    "amount_c",
    "Charge1forCriminalC",
    "Charge2forCriminalC",
    "FineforCriminalC",
    "TotalImprisonmentLengthforCriminalC",
    "SuspendedforCriminalC",
    "Charge1forCriminalD",
    "Charge2forCriminalD",
    "FineforCriminalD",
    "TotalImprisonmentLengthforCriminalD",
    "SuspendedforCriminalD",
    "Charge1forCriminalE",
    "Charge2forCriminalE",
    "FineforCriminalE",
    "TotalImprisonmentLengthforCriminalE",
    "SuspendedforCriminalE",
    "Charge1forCriminalF",
    "Charge2forCriminalF",
    "FineforCriminalF",
    "TotalImprisonmentLengthforCriminalF",
    "SuspendedforCriminalF",
    "Charge3forCriminalC",
]

all_columns = [
    "OriginalLink",
    "CaseNumber",
    "CaseName",
    "Court",
    "Location",
    "CaseType",
    "TrialProcedure",
    "JudgmentDate",
    "PublicationDate",
    "PartiesInvolved",
    "CausesofAction",
    "LegalBasis",
    "FullText",
    "drug_a",
    "amount_a",
    "drug_b",
    "amount_b",
    "ResponseText",
    "Charge1forCriminalA",
    "FineforCriminalA",
    "TotalImprisonmentLengthforCriminalA",
    "SuspendedforCriminalA",
    "Charge2forCriminalA",
    "Charge1forCriminalB",
    "Charge2forCriminalB",
    "FineforCriminalB",
    "TotalImprisonmentLengthforCriminalB",
    "SuspendedforCriminalB",
    "Province",
    "City",
    "District",
    "CourtLevel",
    "Adcode"

]


33
23


check unique values before moving to datacleaning

In [ ]:
import os
import pandas as pd


def align_dataframe(base_path, cause):


    # Using dbutils.fs.ls to list directories/files
    files = dbutils.fs.ls(base_path)

    for sub_file in files:
        if sub_file.name.endswith(".csv"):
            # Reading CSV file into DataFrame
            # Convert to local file path if necessary
            file_path = sub_file.path.replace("dbfs:", "/dbfs")
            df = pd.read_csv(file_path, on_bad_lines='skip')
            
            for x in ['A', 'B', 'C', 'D', 'E', 'F']:
                try:
                    print(df[f'TotalImprisonmentLengthforCriminal{x}'].unique())
                except KeyError:
                    continue
            


base_path = "/mnt/processed_data_criminal_case_analysis/drug_related_DrugTypeAmount_Penalty_Location_March_19"

causes_of_action = ["drug_related"]

for cause in causes_of_action:
    align_dataframe(base_path, cause)

In [ ]:
import os
import pandas as pd


def align_dataframe(base_path, cause):

    # Define the output folder based on the base path and cause
    output_folder = "mnt/processed_data_criminal_case_analysis/drug_related_DrugTypeAmount_Penalty_Location_DataframeAlignment_March_20"

    # Using dbutils.fs.ls to list directories/files
    files = dbutils.fs.ls(base_path)

    for sub_file in files:
        if sub_file.name.endswith(".csv"):
            # Reading CSV file into DataFrame
            # Convert to local file path if necessary
            file_path = sub_file.path.replace("dbfs:", "/dbfs")
            df = pd.read_csv(file_path, on_bad_lines='skip')
            
            #drop columns that are not needed
            df.drop(columns=columns_to_drop, inplace=True, errors='ignore')
            
            # add missing columns
            for column in all_columns:
                if column not in df.columns:
                    df[column] = None
            
            #data cleaning
            # fulltext empty: drop all row
            
            # replace '无期徒刑' with 99999
            df['TotalImprisonmentLength'].replace({'无期徒刑': 99999}, inplace=True)
            
            

            

            # Construct the output path for the enriched CSV file
            output_file_path = 'dbfs:' + \
                os.path.join(
                    output_folder, f"{os.path.basename(sub_file.name)}")

            # Save the processed DataFrame to the new CSV file, ensuring the path is in "/dbfs" format for local IO
            df_enriched.to_csv(output_file_path.replace(
                "dbfs:", "/dbfs"), index=False)


base_path = "/mnt/processed_data_criminal_case_analysis/drug_related_DrugTypeAmount_Penalty_Location_March_19"

causes_of_action = ["drug_related"]

for cause in causes_of_action:
    align_dataframe(base_path, cause)